In [80]:
import pandas as pd
import numpy as np
import numpy
!pip install optuna
import optuna
from numpy import array
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from google.colab import files


In [81]:
#code for MAPE, referred from the url: https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-percentage-error
#'eps' is an arbitrary small yet strictly positive number to avoid undefined results when y is zero.
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    eps=0.01
    for i in range(len(y_true)):
      if y_true[i]==0.00:
        y_true[i]=eps
    return np.mean((np.abs(y_true - y_pred)) / np.abs(y_true)) * 100

In [82]:
x=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
x = x.loc["2016-01-01" :]                         #Since 2016-01-01, 5y(1234rows till 2020-12-31), + year 2021's rows (till 30th of April)
y=x.copy()                                        #deep copy
x.reset_index(drop=True, inplace=True)
x

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,INFY,JSWSTEEL,KOTAKBANK,NESTLEIND,RELIANCE,SHREECEM,TCS,TATASTEEL,TECHM,TITAN,WIPRO
0,850.608887,602.348328,1978.543457,1374.428223,1108.503052,385.106506,528.506653,82.550529,792.340698,1169.114624,227.737686,482.005096,93.927940,725.564270,5311.719238,486.556549,11121.029297,1082.013306,219.908173,461.389832,340.842285,202.975677
1,852.593201,598.059875,1954.307739,1368.077148,1096.297607,385.152100,519.647644,78.515373,794.607056,1130.328369,221.286560,470.513763,95.546753,703.665161,5205.810547,476.948639,10804.255859,1061.057739,219.481003,458.151581,340.212952,203.431641
2,871.807617,592.664673,1940.843384,1365.361816,1094.199951,383.717926,515.715759,80.314178,784.385376,1123.546631,222.282349,468.398560,99.058762,706.109497,5199.016113,481.668732,10867.756836,1051.810791,234.346603,463.031219,336.630768,203.139801
3,858.013977,603.474854,1933.413330,1381.469727,1084.378296,383.080475,517.997192,78.320908,779.852783,1123.453613,216.567245,466.348938,96.781441,701.969116,5251.443359,494.631134,11188.242188,1066.431152,229.604980,465.870178,336.776001,202.483200
4,844.413879,603.380981,1921.943726,1356.088257,1074.604370,375.636505,512.706055,74.528839,758.761841,1095.722656,213.666412,458.259216,93.868484,690.046875,5177.484863,485.550232,10725.758789,1061.796509,213.500595,453.183197,333.096954,200.568176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2557.899902,4736.549805,10001.750000,3526.107910,3784.350098,918.745850,1404.800049,348.350006,2360.649902,2509.800049,591.099976,1343.550049,655.799988,1759.650024,16779.318359,1937.849976,28062.599609,3085.706299,940.750000,962.200012,1479.849976,480.299988
1310,2574.350098,4865.049805,10091.349609,3528.100342,3908.949951,918.795288,1438.699951,366.250000,2379.850098,2518.399902,598.750000,1348.500000,665.900024,1750.300049,16688.214844,1988.650024,28098.550781,3116.754150,977.750000,969.250000,1495.099976,485.049988
1311,2614.550049,5280.899902,10489.299805,3465.881348,3882.600098,913.799988,1476.800049,362.600006,2406.550049,2577.000000,621.349976,1356.000000,662.650024,1811.449951,16543.800781,1997.300049,28687.550781,3108.892822,971.400024,977.400024,1508.849976,489.299988
1312,2613.449951,5484.850098,11176.549805,3456.067871,3910.850098,909.549988,1472.500000,372.149994,2407.600098,2538.850098,621.450012,1356.349976,726.500000,1805.000000,16502.550781,2024.050049,28444.349609,3100.085693,1031.349976,976.900024,1506.800049,489.850006


In [83]:
stonks=[]
for i in x:
  stonks.append(i)
len(stonks)

22

In [84]:
alldata=x   #the original dataset

In [85]:
#timesteps=60                                     #lstm hyperparameters "Subject to be tuned"
#epoch=variable
#batchSize=32
#ineurons=175
#hneurons=187


In [86]:
def create_dataset(dataset, time_step=1):         # convert an array of values into a dataset matrix which will be used to train the lstm model.
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		a = dataset[i:(i+time_step), 0]               #i=0, 0,1,2,3-----(timesteps-1)  -> timesteps
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [87]:
# Static parameters
after2020=len(y.loc["2021-01-01" : ])                    #number of days after 31-12-2020 "automated"
batchSize=88
dropout=0.37

In [88]:
def stacked_lstm_forecast(df1, batchSize, after2020):
  timesteps=60
  scaler=MinMaxScaler(feature_range=(0,1))
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))           #minmax scalar transformation of data

  before_2021_data_length=int(len(df1)-after2020)                 #length of data before 2021
  training_size=int(before_2021_data_length*0.80)                 #80% of training size, refered from Yadav et al (2020) (Science Direct)
  train_data=df1[0:training_size,:]
  test_data=df1[training_size:before_2021_data_length,:1]         #20% of testing data, refered from Yadav et al (2020) (Science Direct)
  inpdata=df1[before_2021_data_length-timesteps:len(df1),:1]      #getting the data from 01-01-2021 onwards


  #reshape into X=t,t+1,t+2,t+3,........t+"timestep-1" and Y=t+"timestep"
  X_train, y_train = create_dataset(train_data, timesteps)
  x_inp, y_inp = create_dataset(inpdata, timesteps)
  x_test, y_test = create_dataset(test_data,timesteps)

  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  x_inp = x_inp.reshape(x_inp.shape[0],x_inp.shape[1] , 1)        #reshape input to be [samples, time steps, features] which is required for LSTM
  x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

  # initialising stacked lstm
  model=Sequential()
  model.add(LSTM(60,return_sequences=True,input_shape=(timesteps,1),activation='tanh', dropout=0.1))
  model.add(LSTM(264,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(80,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(416,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(160,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(472,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(440,activation='tanh', dropout=0.1))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='mean_squared_error',optimizer='adam')

  model.fit(X_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=batchSize,verbose=1)     # training of the model

  test_predict=model.predict(x_test)                    #prediction using test data as input

  #performance metrics between, original test data and predicted test data
  msetst =mean_squared_error(y_test,test_predict)
  rmsetst=math.sqrt(msetst)
  maetst =mean_absolute_error(y_test,test_predict)
  r2tst  =r2_score(y_test,test_predict)
  mapetst=mean_absolute_percentage_error(y_test,test_predict)
  tstlst =[msetst,rmsetst,maetst,r2tst,mapetst]


  #model is trained again on the test data so as to increase the learning (it is often termed as incremental learning)
  #refered from url: https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/#step-2-transforming-the-dataset-for-tensorflow-keras
  #refered from url: https://github.com/keras-team/keras/issues/4446
  model.fit(x_test,y_test,epochs=10,batch_size=batchSize,verbose=1)

  out_predict=model.predict(x_inp)                      #dynamic prediction of the stock's closing price from 01-01-2021 onwards

  #performance metrics between, original data(after 31-12-2020) and dynamically predicted data (after 31-12-2020)
  mseinp =mean_squared_error(y_inp,out_predict)
  rmseinp=math.sqrt(mseinp)
  maeinp =mean_absolute_error(y_inp,out_predict)
  r2inp  =r2_score(y_inp,out_predict)
  mapeinp=mean_absolute_percentage_error(y_inp,out_predict)
  inplst =[mseinp,rmseinp,maeinp,r2inp,mapeinp]


  lst=[]
  for i in out_predict:
    lst.append(i)

  p=train_data.tolist()
  q=test_data.tolist()
  p.extend(q)                                         #appending train and test data to make dataset before 2021 (data till 31-12-2020)
  p.extend(lst)                                       #appending the data, forcasted from 01-01-2021 onwards, to the data till 31-12-2020
  p=scaler.inverse_transform(p).tolist()

  return pd.DataFrame(p), tstlst, inplst
  #returns a dataframe, tstlst => test performance metrics, inplst => forcasted data performance metrics


In [89]:
mtest=[]
mdynamic=[]
fdata=pd.DataFrame()
for i in alldata:                                   # this for loop will be iterated for 42 times i.e. for each column of the original dataset
  temp=alldata[i]
  ftemp,trmse,drmse=stacked_lstm_forecast(temp, batchSize, after2020)    #hyperparameters are provided as input here
  fdata = pd.concat([fdata,ftemp],axis = 1)
  mtest.append(trmse)
  mdynamic.append(drmse)
fdata.columns=stonks

Epoch 1/10
11/11 [==============================] - 33s 940ms/step - loss: 0.0629 - val_loss: 0.3250
Epoch 2/10
11/11 [==============================] - 3s 239ms/step - loss: 0.0521 - val_loss: 0.1449
Epoch 3/10
11/11 [==============================] - 3s 231ms/step - loss: 0.0375 - val_loss: 0.2520
Epoch 4/10
11/11 [==============================] - 3s 230ms/step - loss: 0.0184 - val_loss: 0.1603
Epoch 5/10
11/11 [==============================] - 3s 231ms/step - loss: 0.0145 - val_loss: 0.1179
Epoch 6/10
11/11 [==============================] - 3s 231ms/step - loss: 0.0139 - val_loss: 0.1511
Epoch 7/10
11/11 [==============================] - 3s 231ms/step - loss: 0.0129 - val_loss: 0.1451
Epoch 8/10
11/11 [==============================] - 3s 229ms/step - loss: 0.0128 - val_loss: 0.1382
Epoch 9/10
11/11 [==============================] - 3s 231ms/step - loss: 0.0127 - val_loss: 0.1432
Epoch 10/10
11/11 [==============================] - 2s 228ms/step - loss: 0.0128 - val_loss: 0.134

In [90]:
fdata # dataset with 2021 rows forcasted dynamically

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,INFY,JSWSTEEL,KOTAKBANK,NESTLEIND,RELIANCE,SHREECEM,TCS,TATASTEEL,TECHM,TITAN,WIPRO
0,850.608887,602.348328,1978.543457,1374.428223,1108.503052,385.106506,528.506653,82.550529,792.340698,1169.114624,227.737686,482.005096,93.927940,725.564270,5311.719238,486.556549,11121.029297,1082.013306,219.908173,461.389832,340.842285,202.975677
1,852.593201,598.059875,1954.307739,1368.077148,1096.297607,385.152100,519.647644,78.515373,794.607056,1130.328369,221.286560,470.513763,95.546753,703.665161,5205.810547,476.948639,10804.255859,1061.057739,219.481003,458.151581,340.212952,203.431641
2,871.807617,592.664673,1940.843384,1365.361816,1094.199951,383.717926,515.715759,80.314178,784.385376,1123.546631,222.282349,468.398560,99.058762,706.109497,5199.016113,481.668732,10867.756836,1051.810791,234.346603,463.031219,336.630768,203.139801
3,858.013977,603.474854,1933.413330,1381.469727,1084.378296,383.080475,517.997192,78.320908,779.852783,1123.453613,216.567245,466.348938,96.781441,701.969116,5251.443359,494.631134,11188.242188,1066.431152,229.604980,465.870178,336.776001,202.483200
4,844.413879,603.380981,1921.943726,1356.088257,1074.604370,375.636505,512.706055,74.528839,758.761841,1095.722656,213.666412,458.259216,93.868484,690.046875,5177.484863,485.550232,10725.758789,1061.796509,213.500595,453.183197,333.096954,200.568176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,1986.614160,3241.902346,6627.064108,3615.569325,2775.235424,1046.397339,1420.656729,322.422721,2202.324359,2422.641664,466.208323,919.907369,395.984256,1918.267375,17044.753037,2030.078557,26686.418519,2939.876837,782.473710,923.287178,1464.225431,161.208529
1310,1986.614160,3175.473829,6558.247658,3618.535879,2775.236056,1046.397339,1417.468655,322.797261,2202.477276,2422.121943,466.353869,919.907369,397.760128,1914.238914,17045.124417,2026.924963,26759.984768,2942.081763,787.523444,923.608558,1465.920530,161.208529
1311,1986.614160,3110.231735,6503.807108,3621.393382,2775.236899,1046.397339,1414.202188,323.235891,2202.624825,2421.882983,466.624559,919.907369,399.569175,1910.619951,17045.508344,2023.669298,26810.682794,2944.108100,791.853723,923.852954,1467.222223,161.208529
1312,1986.614160,3049.306522,6450.885521,3624.072282,2775.237532,1046.397339,1410.933632,323.722010,2202.764691,2421.822731,467.072658,919.907369,401.385215,1907.381403,17045.888925,2020.333870,26836.464557,2945.900565,795.485224,923.944135,1468.076032,161.208529


In [91]:
alldata # dataset with original 2021 rows

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,INFY,JSWSTEEL,KOTAKBANK,NESTLEIND,RELIANCE,SHREECEM,TCS,TATASTEEL,TECHM,TITAN,WIPRO
0,850.608887,602.348328,1978.543457,1374.428223,1108.503052,385.106506,528.506653,82.550529,792.340698,1169.114624,227.737686,482.005096,93.927940,725.564270,5311.719238,486.556549,11121.029297,1082.013306,219.908173,461.389832,340.842285,202.975677
1,852.593201,598.059875,1954.307739,1368.077148,1096.297607,385.152100,519.647644,78.515373,794.607056,1130.328369,221.286560,470.513763,95.546753,703.665161,5205.810547,476.948639,10804.255859,1061.057739,219.481003,458.151581,340.212952,203.431641
2,871.807617,592.664673,1940.843384,1365.361816,1094.199951,383.717926,515.715759,80.314178,784.385376,1123.546631,222.282349,468.398560,99.058762,706.109497,5199.016113,481.668732,10867.756836,1051.810791,234.346603,463.031219,336.630768,203.139801
3,858.013977,603.474854,1933.413330,1381.469727,1084.378296,383.080475,517.997192,78.320908,779.852783,1123.453613,216.567245,466.348938,96.781441,701.969116,5251.443359,494.631134,11188.242188,1066.431152,229.604980,465.870178,336.776001,202.483200
4,844.413879,603.380981,1921.943726,1356.088257,1074.604370,375.636505,512.706055,74.528839,758.761841,1095.722656,213.666412,458.259216,93.868484,690.046875,5177.484863,485.550232,10725.758789,1061.796509,213.500595,453.183197,333.096954,200.568176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2557.899902,4736.549805,10001.750000,3526.107910,3784.350098,918.745850,1404.800049,348.350006,2360.649902,2509.800049,591.099976,1343.550049,655.799988,1759.650024,16779.318359,1937.849976,28062.599609,3085.706299,940.750000,962.200012,1479.849976,480.299988
1310,2574.350098,4865.049805,10091.349609,3528.100342,3908.949951,918.795288,1438.699951,366.250000,2379.850098,2518.399902,598.750000,1348.500000,665.900024,1750.300049,16688.214844,1988.650024,28098.550781,3116.754150,977.750000,969.250000,1495.099976,485.049988
1311,2614.550049,5280.899902,10489.299805,3465.881348,3882.600098,913.799988,1476.800049,362.600006,2406.550049,2577.000000,621.349976,1356.000000,662.650024,1811.449951,16543.800781,1997.300049,28687.550781,3108.892822,971.400024,977.400024,1508.849976,489.299988
1312,2613.449951,5484.850098,11176.549805,3456.067871,3910.850098,909.549988,1472.500000,372.149994,2407.600098,2538.850098,621.450012,1356.349976,726.500000,1805.000000,16502.550781,2024.050049,28444.349609,3100.085693,1031.349976,976.900024,1506.800049,489.850006


In [92]:
fdata.shape

(1314, 22)

In [93]:
alldata.shape


(1314, 22)

In [94]:
clm=['MSE','RMSE','MAE','R2','MAPE']
pd.DataFrame(mtest,index=stonks,columns=clm).to_csv('mtest.csv') #metric values saved
files.download('mtest.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [95]:
n50_date=pd.read_csv('n50.csv')
fdata.insert(0,"Date",n50_date['Date'],True)


In [96]:
fdata.reset_index(drop=True, inplace=True)
fdata.to_csv('fdata.csv') 
files.download('fdata.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [97]:
for i in alldata:
  print(alldata[i])

0        850.608887
1        852.593201
2        871.807617
3        858.013977
4        844.413879
           ...     
1309    2557.899902
1310    2574.350098
1311    2614.550049
1312    2613.449951
1313    2536.399902
Name: ASIANPAINT, Length: 1314, dtype: float64
0        602.348328
1        598.059875
2        592.664673
3        603.474854
4        603.380981
           ...     
1309    4736.549805
1310    4865.049805
1311    5280.899902
1312    5484.850098
1313    5451.899902
Name: BAJFINANCE, Length: 1314, dtype: float64
0        1978.543457
1        1954.307739
2        1940.843384
3        1933.413330
4        1921.943726
            ...     
1309    10001.750000
1310    10091.349609
1311    10489.299805
1312    11176.549805
1313    11041.650391
Name: BAJAJFINSV, Length: 1314, dtype: float64
0       1374.428223
1       1368.077148
2       1365.361816
3       1381.469727
4       1356.088257
           ...     
1309    3526.107910
1310    3528.100342
1311    3465.881348
1312    